In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import pickle

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount("drive")

Mounted at drive


In [3]:
# Load data
trainX = np.load("drive/MyDrive/Datasets/lc/v4/trainX.npy").astype(np.float32) / 255.0
trainY = (np.load("drive/MyDrive/Datasets/lc/v4/trainY.npy").astype(np.float32) > 127).astype(np.float32)
testX = np.load("drive/MyDrive/Datasets/lc/v4/testX.npy").astype(np.float32) / 255.0
testY = (np.load("drive/MyDrive/Datasets/lc/v4/testY.npy").astype(np.float32) > 127).astype(np.float32)

In [4]:
# Reshape data
trainX = np.reshape(trainX, (len(trainX), 512, 512, 1))
trainY = np.reshape(trainY, (len(trainY), 512, 512, 1))
testX = np.reshape(testX, (len(testX), 512, 512, 1))
testY = np.reshape(testY, (len(testY), 512, 512, 1))

In [5]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [6]:
# U-Net model
def unet(input_size=(512, 512, 1)):
    inputs = Input(input_size)
    K.set_image_data_format('channels_last')

    # Encoder
    conv1 = Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    conv1 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(pool4)
    conv5 = Conv2D(512, (3, 3), activation="relu", padding="same")(conv5)

    # Decoder
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(conv5), conv4])
    conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(up6)
    conv6 = Conv2D(256, (3, 3), activation="relu", padding="same")(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(conv6), conv3])
    conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(up7)
    conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(conv7), conv2])
    conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(up8)
    conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(conv8), conv1])
    conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(up9)
    conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv9)

    conv10 = Conv2D(1, (1, 1), activation="sigmoid")(conv9)

    return Model(inputs, conv10)

In [7]:
# TPU setup
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [8]:
# Compile model
with strategy.scope():
    model = unet(input_size=(512, 512, 1))
    model.compile(optimizer=Adam(learning_rate=2e-4), loss=dice_coef_loss, metrics=[dice_coef, 'binary_accuracy'])

In [9]:
# Callbacks
checkpoint = ModelCheckpoint("checkpoint-{epoch:03d}-{val_dice_coef:.3f}.hdf5", monitor='val_dice_coef',
                             verbose=1, save_best_only=True, mode='max', save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, verbose=1, min_lr=1e-6)
early_stop = EarlyStopping(monitor="val_loss", patience=30, verbose=1, mode='min')

callbacks = [checkpoint, reduce_lr, early_stop]

In [10]:
# Train the model
history = model.fit(trainX, trainY, batch_size=32, epochs=250, validation_data=(testX, testY), callbacks=callbacks)

Epoch 1/250
169/169 [==============================] - ETA: 0s - loss: -0.0388 - dice_coef: 0.0396 - binary_accuracy: 0.8485
Epoch 1: val_dice_coef improved from -inf to 0.11273, saving model to checkpoint-001-0.113.hdf5
169/169 [==============================] - 246s 1s/step - loss: -0.0388 - dice_coef: 0.0396 - binary_accuracy: 0.8485 - val_loss: -0.0937 - val_dice_coef: 0.1127 - val_binary_accuracy: 0.9980 - lr: 2.0000e-04
Epoch 2/250
169/169 [==============================] - ETA: 0s - loss: -0.2984 - dice_coef: 0.2991 - binary_accuracy: 0.9985
Epoch 2: val_dice_coef improved from 0.11273 to 0.42217, saving model to checkpoint-002-0.422.hdf5
169/169 [==============================] - 53s 311ms/step - loss: -0.2984 - dice_coef: 0.2991 - binary_accuracy: 0.9985 - val_loss: -0.4132 - val_dice_coef: 0.4222 - val_binary_accuracy: 0.9991 - lr: 2.0000e-04
Epoch 3/250
169/169 [==============================] - ETA: 0s - loss: -0.4767 - dice_coef: 0.4772 - binary_accuracy: 0.9992
Epoch 3: v

KeyboardInterrupt: 

In [ ]:
# Save training history
with open('LC_UNet_trainHistory.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# Save the final model
model.save("LC_v4_UNet_cl.h5")

In [ ]:
# Evaluate the model
loss, dice, accuracy = model.evaluate(testX, testY)
print(f"Loss: {loss}, Dice Coefficient: {dice}, Accuracy: {accuracy}")

In [ ]:
# Predictions for metric calculations
predictions = model.predict(testX)
pred_binary = (predictions > 0.5).astype(np.float32)
y_true_flat = testY.flatten()
y_pred_flat = pred_binary.flatten()


In [ ]:
precision = precision_score(y_true_flat, y_pred_flat)
recall = recall_score(y_true_flat, y_pred_flat)
f1 = f1_score(y_true_flat, y_pred_flat)

In [ ]:
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Training and Validation Loss")

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['dice_coef'], label='Dice Coefficient')
plt.plot(history.history['val_dice_coef'], label='Validation Dice Coefficient')
plt.legend()
plt.title("Training and Validation Dice Coefficient")
plt.show()